In [24]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
os.getcwd()
# Any results you write to the current directory are saved as output.

'/home/jayesh/Documents/ML&DL_stuff/NorthEastern_SMILE/recognizing-faces-in-the-wild'

Necassary Imports

In [25]:
!pip install --user git+https://github.com/rcmalli/keras-vggface.git

  Cloning https://github.com/rcmalli/keras-vggface.git to /tmp/pip-req-build-8p4eumn8
  Running command git clone -q https://github.com/rcmalli/keras-vggface.git /tmp/pip-req-build-8p4eumn8
  Stored in directory: /tmp/pip-ephem-wheel-cache-w48h49rz/wheels/36/07/46/06c25ce8e9cd396dabe151ea1d8a2bc28dafcb11321c1f3a6d
Successfully built keras-vggface


In [31]:
import h5py
from collections import defaultdict
from glob import glob
from random import choice, sample

import cv2
import numpy as np
import pandas as pd
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.layers import Input, Dense, Flatten, GlobalMaxPool2D, GlobalAvgPool2D, Concatenate, Multiply, Dropout, Subtract, Add, Conv2D
from keras.models import Model
from keras.optimizers import Adam
#from keras_vggface.utils import preprocess_input
#from keras_vggface.vggface import VGGFace
from keras.applications.imagenet_utils import _obtain_input_shape

ImportError: cannot import name '_obtain_input_shape'

In [4]:
train_file_path = "train_relationships.csv"
train_folders_path = "/train/"
val_famillies = "F09"

In [5]:
all_images = glob(train_folders_path + "*/*/*.jpg")

Validation set consists only family 9

In [6]:
train_images = [x for x in all_images if val_famillies not in x]
val_images = [x for x in all_images if val_famillies in x]

In [7]:
train_person_to_images_map = defaultdict(list)

In [8]:
ppl = [x.split("/")[-3] + "/" + x.split("/")[-2] for x in all_images]

In [9]:
for x in train_images:
    train_person_to_images_map[x.split("/")[-3] + "/" + x.split("/")[-2]].append(x)

val_person_to_images_map = defaultdict(list)

for x in val_images:
    val_person_to_images_map[x.split("/")[-3] + "/" + x.split("/")[-2]].append(x)

relationships = pd.read_csv(train_file_path)
relationships = list(zip(relationships.p1.values, relationships.p2.values))
relationships = [x for x in relationships if x[0] in ppl and x[1] in ppl]

train = [x for x in relationships if val_famillies not in x[0]]
val = [x for x in relationships if val_famillies in x[0]]

In [16]:
def read_img(path):
    img = cv2.imread(path)
    img = np.array(img).astype(np.float)
    return preprocess_input(img, version=2)


def gen(list_tuples, person_to_images_map, batch_size=16):
    ppl = list(person_to_images_map.keys())
    while True:
        batch_tuples = sample(list_tuples, batch_size // 2)
        labels = [1] * len(batch_tuples)
        while len(batch_tuples) < batch_size:
            p1 = choice(ppl)
            p2 = choice(ppl)

            if p1 != p2 and (p1, p2) not in list_tuples and (p2, p1) not in list_tuples:
                batch_tuples.append((p1, p2))
                labels.append(0)

        for x in batch_tuples:
            if not len(person_to_images_map[x[0]]):
                print(x[0])

        X1 = [choice(person_to_images_map[x[0]]) for x in batch_tuples]
        X1 = np.array([read_img(x) for x in X1])

        X2 = [choice(person_to_images_map[x[1]]) for x in batch_tuples]
        X2 = np.array([read_img(x) for x in X2])

        yield [X1, X2], labels


def baseline_model():
    input_1 = Input(shape=(224, 224, 3))
    input_2 = Input(shape=(224, 224, 3))

    base_model = VGGFace(model='resnet50', include_top=False)

    for layer in base_model.layers[:-3]:
        layer.trainable = True

    x1 = base_model(input_1)
    x2 = base_model(input_2)


    merged_add = Add()([x1, x2])
    merged_sub = Subtract()([x1,x2])
    
    merged_add = Conv2D(100 , [1,1] )(merged_add)
    merged_sub = Conv2D(100 , [1,1] )(merged_sub)
    
    merged = Concatenate(axis=-1)([merged_add, merged_sub])
    
    merged = Flatten()(merged)

    merged = Dense(100, activation="relu")(merged)
#     merged = Dropout(0.3)(merged)
#     merged = Dense(25, activation="relu")(merged)
#     merged = Dropout(0.3)(merged)
#     out = Dense(1, activation="sigmoid")(merged)

    model = Model([input_1, input_2])

    model.compile(loss="binary_crossentropy", metrics=['acc'], optimizer=Adam(0.00001))

    model.summary()

    return model


file_path = "vgg_face.h5"

checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

reduce_on_plateau = ReduceLROnPlateau(monitor="val_acc", mode="max", factor=0.1, patience=20, verbose=1)

callbacks_list = [checkpoint, reduce_on_plateau]

model = baseline_model()

NameError: name 'VGGFace' is not defined

In [11]:
model.fit_generator(gen(train, train_person_to_images_map, batch_size=16), use_multiprocessing=True,
                    validation_data=gen(val, val_person_to_images_map, batch_size=16), epochs=130, verbose=1,
                    workers = 4, callbacks=callbacks_list, steps_per_epoch=200, validation_steps=100)

Epoch 1/130


/opt/conda/lib/python3.6/site-packages/keras/engine/training_generator.py:47: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


200/200 [==============================] - 105s 526ms/step - loss: 1.2316 - acc: 0.5394 - val_loss: 0.7537 - val_acc: 0.5994

Epoch 00001: val_acc improved from -inf to 0.59937, saving model to vgg_face.h5
Epoch 2/130
200/200 [==============================] - 78s 390ms/step - loss: 0.8789 - acc: 0.5725 - val_loss: 0.7092 - val_acc: 0.6088

Epoch 00002: val_acc improved from 0.59937 to 0.60875, saving model to vgg_face.h5
Epoch 3/130
200/200 [==============================] - 78s 391ms/step - loss: 0.7836 - acc: 0.5741 - val_loss: 0.6885 - val_acc: 0.6081

Epoch 00003: val_acc did not improve from 0.60875
Epoch 4/130
200/200 [==============================] - 79s 395ms/step - loss: 0.6929 - acc: 0.6081 - val_loss: 0.6612 - val_acc: 0.6044

Epoch 00004: val_acc did not improve from 0.60875
Epoch 5/130
200/200 [==============================] - 79s 395ms/step - loss: 0.7112 - acc: 0.5900 - val_loss: 0.6429 - val_acc: 0.6238

Epoch 00005: val_acc improved from 0.60875 to 0.62375, saving m

In [15]:
!pip3 install --user keras

In [12]:
test_path = "../input/test/"


def chunker(seq, size=32):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))


from tqdm import tqdm

submission = pd.read_csv('../input/sample_submission.csv')

predictions = []

for batch in tqdm(chunker(submission.img_pair.values)):
    X1 = [x.split("-")[0] for x in batch]
    X1 = np.array([read_img(test_path + x) for x in X1])

    X2 = [x.split("-")[1] for x in batch]
    X2 = np.array([read_img(test_path + x) for x in X2])

    pred = model.predict([X1, X2]).ravel().tolist()
    predictions += pred

submission['is_related'] = predictions

submission.to_csv("vgg_face.csv", index=False)

166it [01:22,  1.81it/s]
